Übung: Serverless Computing mit K-native
-----------------------------

**Serverless Computing**, auch bekannt als Function as a Service (FaaS), ist ein modernes Cloud-Computing-Modell, das es Entwicklern ermöglicht, Anwendungen zu erstellen und auszuführen, ohne sich um die Verwaltung von Servern kümmern zu müssen. Bei Serverless-Architekturen werden die Ressourcen dynamisch bereitgestellt und skaliert,

- - -

Zuerst erstellen wir den Kubernetes Namespace

In [ ]:
import os
os.environ['NS_FAAS']='ms-faas'
! kubectl create namespace ${NS_FAAS}
! kubectl label  namespace ${NS_FAAS} istio-injection=enabled

In [ ]:
%%bash
kubectl label namespace ${NS_FAAS} istio-injection=enabled
kubectl apply --namespace ${NS_FAAS} -f https://gitlab.com/ch-mc-b/autoshop-ms/infra/kubernetes-templates/-/raw/main/3-2-0-deployment/catalog-deployment.yaml
kubectl apply --namespace ${NS_FAAS} -f https://gitlab.com/ch-mc-b/autoshop-ms/infra/kubernetes-templates/-/raw/main/3-2-0-deployment/customer-deployment.yaml
kubectl apply --namespace ${NS_FAAS} -f https://gitlab.com/ch-mc-b/autoshop-ms/infra/kubernetes-templates/-/raw/main/3-2-0-deployment/order-deployment.yaml
kubectl apply --namespace ${NS_FAAS} -f https://gitlab.com/ch-mc-b/autoshop-ms/infra/kubernetes-templates/-/raw/main/3-2-0-deployment/webshop-deployment.yaml 
kubectl apply --namespace ${NS_FAAS} -f https://gitlab.com/ch-mc-b/autoshop-ms/infra/kubernetes-templates/-/raw/main/3-2-0-deployment/catalog-service.yaml
kubectl apply --namespace ${NS_FAAS} -f https://gitlab.com/ch-mc-b/autoshop-ms/infra/kubernetes-templates/-/raw/main/3-2-0-deployment/customer-service.yaml
kubectl apply --namespace ${NS_FAAS} -f https://gitlab.com/ch-mc-b/autoshop-ms/infra/kubernetes-templates/-/raw/main/3-2-0-deployment/order-service.yaml
kubectl apply --namespace ${NS_FAAS} -f https://gitlab.com/ch-mc-b/autoshop-ms/infra/kubernetes-templates/-/raw/main/3-2-0-deployment/webshop-service.yaml
kubectl apply --namespace ${NS_FAAS} -f https://gitlab.com/ch-mc-b/autoshop-ms/infra/kubernetes-templates/-/raw/main/3-2-0-deployment/shipment-deployment.yaml
kubectl apply --namespace ${NS_FAAS} -f https://gitlab.com/ch-mc-b/autoshop-ms/infra/kubernetes-templates/-/raw/main/3-2-0-deployment/invoicing-deployment.yaml
kubectl apply --namespace ${NS_FAAS} -f https://gitlab.com/ch-mc-b/autoshop-ms/infra/kubernetes-templates/-/raw/main/3-2-0-deployment/sales-deployment.yaml
kubectl apply --namespace ${NS_FAAS} -f https://gitlab.com/ch-mc-b/autoshop-ms/infra/kubernetes-templates/-/raw/main/3-2-0-deployment/shipment-service.yaml
kubectl apply --namespace ${NS_FAAS} -f https://gitlab.com/ch-mc-b/autoshop-ms/infra/kubernetes-templates/-/raw/main/3-2-0-deployment/invoicing-service.yaml
kubectl apply --namespace ${NS_FAAS} -f https://gitlab.com/ch-mc-b/autoshop-ms/infra/kubernetes-templates/-/raw/main/3-2-0-deployment/sales-service.yaml
kubectl get   --namespace ${NS_FAAS} pods,services

In [ ]:
! echo "http://"$(cat ~/work/server-ip)":"$(kubectl get service --namespace ${NS_FAAS} webshop -o=jsonpath='{ .spec.ports[0].nodePort }')/webshop

Nach einer gewissen Zeit stellen wir fest, dass einige Container immer laufen und Prozesszeit brauchen, aber praktisch nicht verwendet werden. 

Diese sind **Shipment**, **Invoicing** und **Sales**.

Diese wollen wir mittels K-native zu konfigurieren, dass sie erst gestartet werden, wenn sie benötigt werden. 

Deshalb beenden wir diese und starten sie mittels K-native neu.

In [ ]:
%%bash
kubectl delete --namespace ${NS_FAAS} -f https://gitlab.com/ch-mc-b/autoshop-ms/infra/kubernetes-templates/-/raw/main/3-2-0-deployment/shipment-deployment.yaml
kubectl delete --namespace ${NS_FAAS} -f https://gitlab.com/ch-mc-b/autoshop-ms/infra/kubernetes-templates/-/raw/main/3-2-0-deployment/invoicing-deployment.yaml
kubectl delete --namespace ${NS_FAAS} -f https://gitlab.com/ch-mc-b/autoshop-ms/infra/kubernetes-templates/-/raw/main/3-2-0-deployment/sales-deployment.yaml
kubectl delete --namespace ${NS_FAAS} -f https://gitlab.com/ch-mc-b/autoshop-ms/infra/kubernetes-templates/-/raw/main/3-2-0-deployment/shipment-service.yaml
kubectl delete --namespace ${NS_FAAS} -f https://gitlab.com/ch-mc-b/autoshop-ms/infra/kubernetes-templates/-/raw/main/3-2-0-deployment/invoicing-service.yaml
kubectl delete --namespace ${NS_FAAS} -f https://gitlab.com/ch-mc-b/autoshop-ms/infra/kubernetes-templates/-/raw/main/3-2-0-deployment/sales-service.yaml
kubectl get   --namespace ${NS_FAAS} pods,services

Dann können wir die Container im Serverless Modus starten

In [ ]:
%%bash
kn service create invoicing --image registry.gitlab.com/ch-mc-b/autoshop-ms/app/backoffice/invoicing:4.0.0 --port 8080 --namespace ${NS_FAAS}
kn service create shipment  --image registry.gitlab.com/ch-mc-b/autoshop-ms/app/backoffice/shipment:4.0.0  --port 8080 --namespace ${NS_FAAS}
kn service create sales  --image registry.gitlab.com/ch-mc-b/autoshop-ms/app/management/sales:3.2.0  --port 8080 --namespace ${NS_FAAS}

Zuerst Überprüfen wir ob die Services "Ready" sind

In [ ]:
! kn service ls --namespace ${NS_FAAS} 
! kubectl --namespace ${NS_FAAS} get pods -l serving.knative.dev/configuration 
! # kubectl --namespace ${NS_FAAS} get services -l serving.knative.dev/configuration 

In [ ]:
%%bash
PORT=$(kubectl get service --namespace kourier-system kourier -o=jsonpath='{ .spec.ports[0].nodePort }')
curl -H "Host: shipment.${NS_FAAS}.$(cat ~/work/server-ip)-edutbz.com" http://localhost:${PORT}/shipment/api
curl -H "Host: invoicing.${NS_FAAS}.$(cat ~/work/server-ip)-edutbz.com" http://localhost:${PORT}/invoicing/api
curl -H "Host: sales.${NS_FAAS}.$(cat ~/work/server-ip)-edutbz.com" http://localhost:${PORT}/sales

Um die Services via Browser anzusprechen ist im Windows die Datei `C:\Windows\System32\Drivers\etc\hosts` um folgende Einträge zu ergänzen.

In [ ]:
%%bash
PORT=$(kubectl get service --namespace kourier-system kourier -o=jsonpath='{ .spec.ports[0].nodePort }')
echo $(hostname -I | cut '-d ' -f1) shipment.${NS_FAAS}.$(cat ~/work/server-ip)-edutbz.com
echo $(hostname -I | cut '-d ' -f1) invoicing.${NS_FAAS}.$(cat ~/work/server-ip)-edutbz.com
echo $(hostname -I | cut '-d ' -f1) sales.${NS_FAAS}.$(cat ~/work/server-ip)-edutbz.com

echo ""
echo "Browser URLs"
echo http://shipment.${NS_FAAS}.$(cat ~/work/server-ip)-edutbz.com:${PORT}/shipment
echo http://invoicing.${NS_FAAS}.$(cat ~/work/server-ip)-edutbz.com:${PORT}/invoicing
echo http://sales.${NS_FAAS}.$(cat ~/work/server-ip)-edutbz.com:${PORT}/sales


Im Dashboard können wir das Verhalten beobachten. Nach einer gewissen Zeit werden die Pods **Shipment**, **Invoicing** und **Sales** beendet. Durch drücken von Refresh im Browser werden diese automatisch wieder gestartet.

In [ ]:
%%bash
echo "https://"$(cat ~/work/server-ip)":30443"

## Lasttest

Um die Verbindungen sichtbar zu machen, erzeugen wir ein wenig Traffic.

Dazu verwenden wir [hey](https://github.com/rakyll/hey) welche gleichzeitige parallele Request unterstützt.

In [ ]:
%%bash
PORT=$(kubectl get service --namespace kourier-system kourier -o=jsonpath='{ .spec.ports[0].nodePort }')
echo "shipment.${NS_FAAS}.$(cat ~/work/server-ip)-edutbz.com"
hey -z 20s -c 50 -H "Host: shipment.${NS_FAAS}.$(cat ~/work/server-ip)-edutbz.com" http://localhost:${PORT}/shipment

- - -

Aufräumen


In [ ]:
! # kubectl delete pod --all --namespace ${NS-FAAS} --grace-period=0 --force
! kn service delete shipment  --namespace ${NS_FAAS}
! kn service delete invoicing --namespace ${NS_FAAS}
! kn service delete sales --namespace ${NS_FAAS}
! kubectl delete namespace ${NS_FAAS}